In [ ]:
import requests
import json
import time
import random
from datetime import datetime
import csv
endpointurl = "https://kr44v8tqe0.execute-api.eu-west-3.amazonaws.com/dev/ingest"

key = "" #Aquí iría la API Key proporcionada para la autenticación
# Intervalo de envío de datos en segundos
Frecuencia_Mensajes = 2
total_mensajes_enviados = 0
mensajes_fallidos = 0
tiempos_latencia = []
coste_por_hora_simulado = 0.05  # Coste estimado por hora de ejecución de este simulador (en USD)
start_time_simulacion = 0 # Inicializamos a 0, se establecerá en main()
# --- Configuración CSV ---
CSV_FILENAME = "telemetry_data.csv"
CSV_HEADERS = ["vanID", "vanTimestamp", "lat", "lon", "speed", "clientID", "sendTime", "status_code", "latency_api", "error_message"] # Nuevos encabezados
# --- Funciones de simulación de datos ---
# Función para inicializar el archivo CSV con los encabezados
def initialize_csv():
    try:
        with open(CSV_FILENAME, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(CSV_HEADERS)
        print(f"Archivo CSV '{CSV_FILENAME}' inicializado con encabezados.")
    except Exception as e:
        print(f"Error al inicializar el archivo CSV: {e}")

# Función para añadir una fila de datos al CSV
def append_to_csv(data_row):
    try:
        with open(CSV_FILENAME, 'a', newline='') as csvfile: # 'a' para añadir al final
            writer = csv.writer(csvfile)
            writer.writerow(data_row)
    except Exception as e:
        print(f"Error al escribir en el archivo CSV: {e}")

def telemetrydata():
    van_id = "van016"
    now = datetime.now()
    vanTimestamp = now.strftime("%H:%M:%S")
    lat = str(round(random.uniform(-90.0, 90.0), 4))
    lon = str(round(random.uniform(-180.0, 180.0), 4))
    speed = str(random.randint(0, 120))
    client_id = "client002"

    data = {
        "vanID": van_id,
        "vanTimestamp": vanTimestamp,
        "lat": lat,
        "lon": lon,
        "speed": speed,
        "clientID": client_id,
        "sendTime": time.time()
    }
    return data

def senddata(data):
    global total_mensajes_enviados, mensajes_fallidos, tiempos_latencia

    # Preparar la fila base para el CSV con los datos generados
    csv_row_base = [
        data["vanID"],
        data["vanTimestamp"],
        data["lat"],
        data["lon"],
        data["speed"],
        data["clientID"],
        data["sendTime"]
    ]

    headers = {
        "Content-type": "application/json",
         "x-api-key": key
    }

    start_time_request = time.time()

    status_code = "N/A"
    latency_api = "N/A"
    error_message = ""

    try:
        response = requests.post(endpointurl, data=json.dumps(data), headers=headers)
        end_time_request = time.time()

        status_code = response.status_code
        response.raise_for_status()

        print(f"Datos enviados con éxito. Estado: {response.status_code}")
        total_mensajes_enviados += 1

        # Latencia aproximada del envío de la API (desde que se creó el dato hasta que la API respondió)
        latency_api = end_time_request - data["sendTime"]
        tiempos_latencia.append(latency_api)

    except requests.exceptions.RequestException as e:
        end_time_request = time.time()
        print(f"Error al enviar los datos: {e}")
        error_message = str(e)
        mensajes_fallidos += 1
        # Si hubo un error en la request, la latencia es hasta que el error fue detectado
        try:
            latency_api = end_time_request - data["sendTime"]
        except KeyError:
            latency_api = "N/A - sendTime missing" # En caso de que sendTime falte por algún motivo

        if hasattr(e, 'response') and e.response is not None:
            status_code = e.response.status_code

    finally:
        # Añadir la información del resultado al CSV
        csv_row = csv_row_base + [status_code, latency_api, error_message]
        append_to_csv(csv_row)

def print_metrics():
    if start_time_simulacion == 0:
        print("\n--- ERROR: Tiempo de inicio de simulación no establecido. No se pueden calcular métricas de coste. ---")
        return

    print("\n--- Métricas del Simulador ---")
    print(f"Total de mensajes enviados con éxito: {total_mensajes_enviados}")
    print(f"Mensajes fallidos (errores de envío): {mensajes_fallidos}")

    # Pérdida de mensajes
    total_intentos = total_mensajes_enviados + mensajes_fallidos
    if total_intentos > 0:
        porcentaje_perdida = (mensajes_fallidos / total_intentos) * 100
        print(f"Pérdida de mensajes: {porcentaje_perdida:.2f}%")
    else:
        print("Pérdida de mensajes: 0.00% (no se han enviado mensajes)")

    # Latencia extremo a extremo (aproximación)
    if tiempos_latencia:
        latencia_media = sum(tiempos_latencia) / len(tiempos_latencia)
        latencia_maxima = max(tiempos_latencia)
        latencia_minima = min(tiempos_latencia)
        print(f"Latencia (envío a respuesta API) - Media: {latencia_media:.4f}s, Máxima: {latencia_maxima:.4f}s, Mínima: {latencia_minima:.4f}s")
        print("(Nota: Esta es la latencia desde la generación del dato hasta la respuesta de la API. La latencia 'extremo a extremo' hasta el dashboard requeriría instrumentación adicional en el dashboard para enviar la hora de recepción.)")
    else:
        print("No hay datos de latencia disponibles (no se han enviado mensajes con éxito).")

    # Coste por hora de pruebas
    # Aquí es una estimación simple basada en el tiempo de ejecución del script
    tiempo_ejecucion_segundos = time.time() - start_time_simulacion
    tiempo_ejecucion_horas = tiempo_ejecucion_segundos / 3600
    coste_actual = tiempo_ejecucion_horas * coste_por_hora_simulado
    print(f"Coste estimado de esta sesión de pruebas (a {coste_por_hora_simulado:.2f} USD/hora): {coste_actual:.4f} USD")
    print("(Este coste es una estimación del 'hardware' que ejecuta este simulador. El coste real de la infraestructura del backend puede ser diferente.)")
    print("-----------------------------\n")

def main():
    global start_time_simulacion
    start_time_simulacion = time.time()

    # Inicializar el archivo CSV antes de empezar el bucle
    initialize_csv()

    print("Iniciando simulador de dispositivo de envío...")
    print(f"Endpoint configurado: {endpointurl}")
    print(f"API Key en uso: {key}")
    print(f"Intervalo de envío: {Frecuencia_Mensajes} segundos.")

    try:
        while True:
            telemetry_data = telemetrydata()
            print(f"\nGenerando datos: {json.dumps(telemetry_data, indent=2)}")

            senddata(telemetry_data)

            time.sleep(Frecuencia_Mensajes)

    except KeyboardInterrupt:
        print("\nSimulador detenido por el usuario. Calculando métricas finales...")
    except Exception as e:
        print(f"Ocurrió un error inesperado en el bucle principal: {e}. Calculando métricas finales...")
    finally:
        print_metrics()

if __name__ == "__main__":
    main()

Archivo CSV 'telemetry_data.csv' inicializado con encabezados.
Iniciando simulador de dispositivo de envío...
Endpoint configurado: https://kr44v8tqe0.execute-api.eu-west-3.amazonaws.com/dev/ingest
Intervalo de envío: 2 segundos.

Generando datos: {
  "vanID": "van016",
  "vanTimestamp": "09:29:16",
  "lat": "-84.4903",
  "lon": "-90.0836",
  "speed": "41",
  "clientID": "client002",
  "sendTime": 1761211756.8699632
}
Error al enviar los datos: 403 Client Error: Forbidden for url: https://kr44v8tqe0.execute-api.eu-west-3.amazonaws.com/dev/ingest

Generando datos: {
  "vanID": "van016",
  "vanTimestamp": "09:29:19",
  "lat": "-56.9469",
  "lon": "175.8304",
  "speed": "91",
  "clientID": "client002",
  "sendTime": 1761211759.698486
}
Error al enviar los datos: 403 Client Error: Forbidden for url: https://kr44v8tqe0.execute-api.eu-west-3.amazonaws.com/dev/ingest

Simulador detenido por el usuario. Calculando métricas finales...

--- Métricas del Simulador ---
Total de mensajes enviados c